In [1]:
from pathlib import Path
import sys
import os
import cobra
import matplotlib.pyplot as plt
from cobra import Reaction, Metabolite, Model
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
import pandas as pd
import numpy as np
import math 
import time
import warnings

In [2]:
path_root = "../src"  # Change path according to your PyCoMo location
sys.path.append(str(path_root))
import pycomo

## Community toy model ##
This simple community toy model consists of two identical community members. Each member has a two step conversion of the starting metabolite A. The products of these conversions (B and C) can be secreted and taken up from the medium, thus allowing exchange and forming a cycle. The members also have a biomass reaction with substrates C and D (an additional substrate).

In [3]:
model_single = Model()
model_single.add_metabolites([Metabolite(i) for i in "ABCD"])

for met in model_single.metabolites:
    met.compartment = "c"

model_single.add_reactions([Reaction(i) for i in ["EX_A", "EX_B", "EX_C", "EX_D", "bio", "v1", "v2"]])

model_single.reactions.EX_A.add_metabolites({"A": 1})
model_single.reactions.EX_B.add_metabolites({"B": 1})
model_single.reactions.EX_C.add_metabolites({"C": 1})
model_single.reactions.EX_D.add_metabolites({"D": 1})
model_single.reactions.bio.add_metabolites({"C": -1, "D": -1})

model_single.reactions.v1.add_metabolites({"A": -1, "B": 1})
model_single.reactions.v2.add_metabolites({"B": -1, "C": 1})

model_single.reactions.EX_B.lower_bound = -500
model_single.reactions.EX_C.lower_bound = -500
model_single.reactions.v2.lower_bound = -1000

model_single.objective = 'bio'

### Constructing the community model ###

In [4]:
single_org_models_toy = []
for name in ["model_a", "model_b"]:
    print(name)
    single_org_model = pycomo.SingleOrganismModel(model_single, name)
    single_org_models_toy.append(single_org_model)

model_a
model_b


In [5]:
community_name = "toy_com"
com_model_obj_toy = pycomo.CommunityModel(single_org_models_toy, community_name)
com_model_obj_toy.convert_to_fixed_abundance()

No community model generated yet. Generating now:
Note: no products in the objective function, adding biomass to it.


Ignoring reaction 'EX_A_medium' since it already exists.
Ignoring reaction 'EX_B_medium' since it already exists.
Ignoring reaction 'EX_C_medium' since it already exists.
Ignoring reaction 'EX_D_medium' since it already exists.


Note: no products in the objective function, adding biomass to it.
Generated community model.


In [6]:
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()

Name,toy_com
Memory address,0x01932a3834f0
Number of metabolites,40
Number of reactions,47
Number of groups,0
Objective expression,1.0*community_biomass - 1.0*community_biomass_reverse_44dc1
Compartments,"model_a_c, model_a_medium, medium, fraction_reaction, model_b_c, model_b_medium"


In [7]:
com_model_obj_toy.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
A_medium,EX_A_medium,1000,0,0.00%
D_medium,EX_D_medium,1000,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
cpd11416_medium,community_biomass,-1000,0,0.00%


### Loops in the toy model ###

In [8]:
com_model_obj_toy.get_loops()

,reaction,min_flux,max_flux
0,model_a_TP_B_model_a_c,-500.0,500.0
1,model_a_TP_C_model_a_c,-500.0,500.0
2,model_a_v2_model_a_c,-500.0,500.0
3,model_b_TP_B_model_b_c,-500.0,500.0
4,model_b_TP_C_model_b_c,-500.0,500.0
5,model_b_v2_model_b_c,-500.0,500.0


## FVA examples ##

### FVA with loops ###
Normal fva will include the loops including metabolites B and C, reaction v2 and the transporters of B and C in the solutions.

The following scenarios and correct outcomes will be tested on a :
 - No medium: no reaction can carry flux
 - 100% objective value: reaction v2 carries maximum flux (500; = 1000 * 0.5 as normalized to equal abundance) and transport reactions of B and C are inactive
 - 0% objective value: reactions v2 and transporters of B and C carry maximum flux (250, reverse also for transporters)
 - 80% objective value: reaction v2 carries flux between 300 and 500. Transporter of B can be active between -200 and 200, transporter of C between -100 and 100. This solution arises by one member giving 200 of B to the other (keeping 300), which converts B to C with maximum flux of 500, then gives back 100 of C while keeping the required 400. This results in a biomass reaction of 400 for both (80% of 500). 

#### COBRApy fva ####
The loopless version of COBRApy cannot be used on PyCoMo community models, due to their bound-free reaction structure. The resulting solutions include the loop:

In [9]:
%%time
with com_model_obj_toy.model:
    com_model_obj_toy.medium = {}
    com_model_obj_toy.apply_medium()
    fva = cobra.flux_analysis.flux_variability_analysis(com_model_obj_toy.model, 
                                                        com_model_obj_toy.model.reactions, 
                                                        fraction_of_optimum=0., 
                                                        loopless=True)
fva

CPU times: total: 78.1 ms
Wall time: 8.29 s


,minimum,maximum
model_a_TP_A_model_a_c,0.0,0.0
model_a_TP_B_model_a_c,-250.0,250.0
model_a_TP_C_model_a_c,-250.0,250.0
model_a_TP_D_model_a_c,0.0,0.0
model_a_bio,0.0,0.0
model_a_v1_model_a_c,0.0,0.0
model_a_v2_model_a_c,-250.0,250.0
EX_A_medium,0.0,0.0
EX_B_medium,0.0,0.0
EX_C_medium,0.0,0.0


#### PyCoMo fva with loops ####
The PyCoMo wrapper for fva will also include loops, when loopless mode is not activated

In [10]:
%%time
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()
with com_model_obj_toy.model:
    com_model_obj_toy.medium = {}
    com_model_obj_toy.apply_medium()
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, fraction_of_optimum=1., loopless=False)
fva

CPU times: total: 125 ms
Wall time: 8.16 s


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_c,model_a_TP_A_model_a_c,0.0,0.0
model_a_TP_B_model_a_c,model_a_TP_B_model_a_c,-250.0,250.0
model_a_TP_C_model_a_c,model_a_TP_C_model_a_c,-250.0,250.0
model_a_TP_D_model_a_c,model_a_TP_D_model_a_c,0.0,0.0
EX_A_medium,EX_A_medium,0.0,0.0
EX_B_medium,EX_B_medium,0.0,0.0
EX_C_medium,EX_C_medium,0.0,0.0
EX_D_medium,EX_D_medium,0.0,0.0
model_a_to_community_biomass,model_a_to_community_biomass,0.0,0.0
model_b_TP_A_model_b_c,model_b_TP_A_model_b_c,0.0,0.0


#### PyCoMo loopless fva ####
The following examples show that the loopless fva implemented in PyCoMo leads to the correct solutions in the 4 test cases (no medium, 100% objective value, 0% objective value, 80% objective value)

In [11]:
%%time
with com_model_obj_toy.model:
    com_model_obj_toy.medium = {}
    com_model_obj_toy.apply_medium()
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, fraction_of_optimum=0., loopless=True)
fva

CPU times: total: 375 ms
Wall time: 365 ms


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_c,model_a_TP_A_model_a_c,0.0,0.0
model_a_TP_B_model_a_c,model_a_TP_B_model_a_c,0.0,0.0
model_a_TP_C_model_a_c,model_a_TP_C_model_a_c,0.0,0.0
model_a_TP_D_model_a_c,model_a_TP_D_model_a_c,0.0,0.0
EX_A_medium,EX_A_medium,0.0,0.0
EX_B_medium,EX_B_medium,0.0,0.0
EX_C_medium,EX_C_medium,0.0,0.0
EX_D_medium,EX_D_medium,0.0,0.0
model_a_to_community_biomass,model_a_to_community_biomass,0.0,0.0
model_b_TP_A_model_b_c,model_b_TP_A_model_b_c,0.0,0.0


In [12]:
%%time
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()
with com_model_obj_toy.model:
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, fraction_of_optimum=1., loopless=True)
fva

CPU times: total: 312 ms
Wall time: 335 ms


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_c,model_a_TP_A_model_a_c,500.0,500.0
model_a_TP_B_model_a_c,model_a_TP_B_model_a_c,0.0,0.0
model_a_TP_C_model_a_c,model_a_TP_C_model_a_c,0.0,0.0
model_a_TP_D_model_a_c,model_a_TP_D_model_a_c,500.0,500.0
EX_A_medium,EX_A_medium,-1000.0,-1000.0
EX_B_medium,EX_B_medium,0.0,0.0
EX_C_medium,EX_C_medium,0.0,0.0
EX_D_medium,EX_D_medium,-1000.0,-1000.0
model_a_to_community_biomass,model_a_to_community_biomass,500.0,500.0
model_b_TP_A_model_b_c,model_b_TP_A_model_b_c,500.0,500.0


In [13]:
%%time
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()
with com_model_obj_toy.model:
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, fraction_of_optimum=0., loopless=True)
fva

CPU times: total: 328 ms
Wall time: 336 ms


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_c,model_a_TP_A_model_a_c,0.0,500.0
model_a_TP_B_model_a_c,model_a_TP_B_model_a_c,-250.0,250.0
model_a_TP_C_model_a_c,model_a_TP_C_model_a_c,-250.0,250.0
model_a_TP_D_model_a_c,model_a_TP_D_model_a_c,0.0,500.0
EX_A_medium,EX_A_medium,-1000.0,0.0
EX_B_medium,EX_B_medium,0.0,500.0
EX_C_medium,EX_C_medium,0.0,500.0
EX_D_medium,EX_D_medium,-1000.0,0.0
model_a_to_community_biomass,model_a_to_community_biomass,0.0,500.0
model_b_TP_A_model_b_c,model_b_TP_A_model_b_c,0.0,500.0


In [14]:
%%time
com_model_obj_toy.medium = {'EX_A_medium': 1000.0, 'EX_D_medium': 1000.0}
com_model_obj_toy.apply_medium()
with com_model_obj_toy.model:
    fva = com_model_obj_toy.run_fva(reactions=com_model_obj_toy.model.reactions, fraction_of_optimum=0.8, loopless=True)
fva

CPU times: total: 344 ms
Wall time: 350 ms


,reaction_id,min_flux,max_flux
model_a_TP_A_model_a_c,model_a_TP_A_model_a_c,300.0,500.0
model_a_TP_B_model_a_c,model_a_TP_B_model_a_c,-200.0,200.0
model_a_TP_C_model_a_c,model_a_TP_C_model_a_c,-100.0,100.0
model_a_TP_D_model_a_c,model_a_TP_D_model_a_c,400.0,500.0
EX_A_medium,EX_A_medium,-1000.0,-800.0
EX_B_medium,EX_B_medium,0.0,200.0
EX_C_medium,EX_C_medium,0.0,200.0
EX_D_medium,EX_D_medium,-1000.0,-800.0
model_a_to_community_biomass,model_a_to_community_biomass,400.0,500.0
model_b_TP_A_model_b_c,model_b_TP_A_model_b_c,300.0,500.0
